Here we try to predict the whether a given person is diabetic or non-diabetic by creating an ML model, precisely we use Support Vector Machine classifier, which is a Supervised ML based algorithm

import the dependencies

In [2]:
import numpy as np # for numpy arrays
import pandas as pd # for structuring the given data into tabular form

from sklearn.preprocessing import StandardScaler # for standard scaling in order to make the values within the same range
from sklearn.model_selection import train_test_split # spliting the dataset into training and testing data
from sklearn import svm # for classification model
from sklearn.metrics import accuracy_score # predicting our accuracy

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


data collection
<p>PIMA diabetes dataset</p>

In [4]:
diabetes_dataset = pd.read_csv("/content/drive/MyDrive/self_projects/diabetes_prediction/diabetes.csv")


In [5]:
diabetes_dataset.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1

EDA (Exploratory Data Analysis)

In [9]:
diabetes_dataset.shape

(768, 9)

In [12]:
diabetes_dataset.iloc[:,-1].value_counts() # or you can use diabetes_dataset["Outcome"].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [6]:
diabetes_dataset.describe()

Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    31.992578                  0.471876   33.240885    0.348958  
std      7.884160                  0.331329   11.760232    0.476951  
min      0.000000                  0.078000   21.000000    0.000000  
25%     27.300000                  0.243750   24.000000    0.000000  
50%     32.000000                  0.372500   29.000000    0.000000  
75%     36.600000                  0.626250   41.000000    1.000000  
max     67.100000                  2.420000   81.000000    1.000000

In [7]:
diabetes_dataset.groupby("Outcome").mean()

Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
Outcome                                                                      
0           3.298000  109.980000      68.184000      19.664000   68.792000   
1           4.865672  141.257463      70.824627      22.164179  100.335821   

               BMI  DiabetesPedigreeFunction        Age  
Outcome                                                  
0        30.304200                  0.429734  31.190000  
1        35.142537                  0.550500  37.067164

splitting the data

In [8]:
input_features = diabetes_dataset.drop(columns = "Outcome", axis = 1)
labelled_data = diabetes_dataset["Outcome"]

In [13]:
input_features

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  
2                       0.672   32  
3                       0.167   21  
4                       2.288   33  
..                        ...  ...  
763                     0.171   63  
764                     0.340   27  
765                     0.245   30  
766                     0.349   47  
767                     0.315   23  

[768 rows x 8 columns]

In [15]:
labelled_data.values # .values converts it into numpy array

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

standardize the data

In [16]:
scaler = StandardScaler()

In [17]:
scaler.fit(input_features)

StandardScaler()

In [18]:
standard_data = scaler.transform(input_features)

In [19]:
input_features = standard_data

In [20]:
print(input_features) # in the range of (-1,1)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


train test split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(input_features, labelled_data, test_size = 0.2, stratify = labelled_data, random_state = 2)

In [22]:
input_features.shape, X_train.shape, X_test.shape

((768, 8), (614, 8), (154, 8))

In [23]:
labelled_data.shape, y_train.shape, y_test.shape

((768,), (614,), (154,))

model creation for classification using Support Vector Machine Classifier

In [24]:
classifier = svm.SVC(kernel = "linear")

In [25]:
classifier.fit(X_train, y_train) # model is created

SVC(kernel='linear')

model prediction

In [27]:
# for train dataset
x_train_predict = classifier.predict(X_train)

In [28]:
print("accuracy score = ", accuracy_score(x_train_predict, y_train))

accuracy score =  0.7866449511400652


In [29]:
# for test dataset
x_test_predict = classifier.predict(X_test)

In [30]:
print("accuracy score = ", accuracy_score(x_test_predict, y_test))

accuracy score =  0.7727272727272727


we can see that overfitting is not happening, coz the accuracy score for both train and test data set are more or less the same

making a predictive system

In [33]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# converting the input data (tuple) into numpy array
input_data_as_np = np.asarray(input_data)

# reshaping the numpy array as it has only 1 row
input_data_reshaped = input_data_as_np.reshape(1,-1)

# standardize the input values
std_input_data = scaler.transform(input_data_reshaped)
#print(std_input_data)

# prediction
prediction = classifier.predict(std_input_data)
print(prediction)

if prediction[0] == 0:
  print("The person is non-diabetic")
else:
  print("The person is diabetic")



[1]
The person is diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
